In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Carregar modelo salvo em formato .keras
model = load_model('model.keras')

# Mapeamento das classes (ajuste conforme seu dataset)


datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
    'dataset/treino',
    target_size=(299, 299),
    batch_size=8,
    class_mode='categorical'
)

# Recuperar os nomes das classes
classes = list(train_generator.class_indices.keys())
print(classes)


# Carregar classificador de rosto do OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Carregar imagem
img = cv2.imread('imgtest/bbb.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detectar rostos
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

for (x, y, w, h) in faces:
    # Recortar o rosto detectado
    face = img[y:y+h, x:x+w]
    face_resized = cv2.resize(face, (299, 299))  # InceptionResNetV2 = 299x299
    face_array = image.img_to_array(face_resized) / 255.0
    face_array = np.expand_dims(face_array, axis=0)

    # Fazer predição
    pred = model.predict(face_array)
    label = classes[np.argmax(pred)]
    confidence = np.max(pred)

    # Desenhar o retângulo e o nome
    color = (0, 255, 0) if confidence > 0.6 else (0, 0, 255)
    cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
    cv2.putText(img, f"{label} ({confidence:.2f})", (x, y-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# Exibir resultado
cv2.imshow('Reconhecimento Facial', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Found 15 images belonging to 3 classes.
['leonard', 'outros', 'sheldon']
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
